# Ensemble Learning

## Initial Imports

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder

In [26]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [27]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
#droping irrelevant columns
df.drop(columns=['issue_d','pymnt_plan','recoveries','collection_recovery_fee','next_pymnt_d','acc_now_delinq','num_tl_120dpd_2m','num_tl_30dpd','tax_liens','hardship_flag','debt_settlement_flag'],inplace=True)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,dti,delinq_2yrs,inq_last_6mths,...,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,27.24,0.0,0.0,...,8.0,0.0,3.0,85.7,100.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,20.23,0.0,0.0,...,17.0,0.0,6.0,91.2,50.0,1.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,24.26,0.0,0.0,...,8.0,0.0,4.0,66.7,50.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,31.44,0.0,1.0,...,10.0,0.0,1.0,100.0,50.0,1.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,18.76,0.0,1.0,...,14.0,0.0,1.0,100.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0


In [28]:
# Create the LabelEncoder instance
le = LabelEncoder()

In [29]:
#Encoding data using LabelEncoder

#loan_status column
le.fit(df['loan_status'])
df['loan_status'] = le.transform(df['loan_status'])

#application_type column
le.fit(df['application_type'])
df['application_type'] = le.transform(df['application_type'])

le.fit(df['initial_list_status'])
df['initial_list_status'] = le.transform(df['initial_list_status'])

In [30]:
#Encoding data using get_dummies()

#home_ownership and verification_status columns
df = pd.get_dummies(df, columns=['home_ownership', 'verification_status'], drop_first=True)
df

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified
0,10500.0,0.1719,375.35,66000.0,1,27.24,0.0,0.0,8.0,0.0,...,0.0,65687.0,38199.0,2000.0,61987.0,0,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,1,20.23,0.0,0.0,17.0,1.0,...,1.0,271427.0,60641.0,41200.0,49197.0,1,0,0,0,1
2,20000.0,0.2000,529.88,56000.0,1,24.26,0.0,0.0,8.0,0.0,...,0.0,60644.0,45684.0,7500.0,43144.0,1,0,0,0,1
3,10000.0,0.1640,353.55,92000.0,1,31.44,0.0,1.0,10.0,1.0,...,1.0,99506.0,68784.0,19700.0,76506.0,0,0,1,0,1
4,22000.0,0.1474,520.39,52000.0,1,18.76,0.0,1.0,14.0,0.0,...,0.0,219750.0,25919.0,27600.0,20000.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,1,9.60,0.0,0.0,9.0,0.0,...,0.0,20625.0,6798.0,11300.0,5425.0,0,0,1,1,0
68813,12000.0,0.2727,368.37,63000.0,1,29.07,0.0,0.0,8.0,0.0,...,0.0,87939.0,60350.0,13500.0,62939.0,0,0,1,0,0
68814,5000.0,0.1992,185.62,52000.0,1,14.86,0.0,0.0,5.0,1.0,...,1.0,30592.0,18611.0,3600.0,18492.0,1,0,0,1,0
68815,40000.0,0.0646,1225.24,520000.0,1,9.96,0.0,1.0,21.0,0.0,...,0.0,1033574.0,95958.0,100800.0,78634.0,1,0,0,0,1


## Split the Data into Training and Testing

In [31]:

# Create our features
X = df.copy()
X.drop(columns='loan_status', inplace=True)


# Create our target
y = df['loan_status']


In [32]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.125972,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.526309,0.106747,0.357659,0.373992,0.148001
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.336732,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.499311,0.308793,0.479314,0.483865,0.355104
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,1.000000,0.000000,0.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,1.000000,0.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,4.000000,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,1.000000,1.000000,1.000000,1.000000,1.000000


In [33]:
# Check the balance of our target values
Counter(y)

Counter({1: 68470, 0: 347})

In [34]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [35]:
from sklearn.preprocessing import StandardScaler
# Create the StandardScaler instance
data_scaler = StandardScaler()

In [36]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [37]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [38]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train_scaled, y_train)
y_pred = brf.predict(X_test_scaled)


In [39]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.7100808449647584

In [40]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   59,    42],
       [ 2805, 14299]])

In [41]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.58      0.84      0.04      0.70      0.48       101
          1       1.00      0.84      0.58      0.91      0.70      0.50     17104

avg / total       0.99      0.83      0.59      0.90      0.70      0.50     17205



In [42]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
importances_df = pd.DataFrame(sorted(zip(importances, X.columns), reverse=True))
importances_df[:10]

,0,1
0,0.068368,total_rec_prncp
1,0.063592,total_rec_int
2,0.059355,last_pymnt_amnt
3,0.056751,total_pymnt_inv
4,0.054577,total_pymnt
5,0.034526,int_rate
6,0.021395,mths_since_recent_inq
7,0.018668,dti
8,0.018073,installment
9,0.017035,max_bal_bc


### Easy Ensemble Classifier

In [43]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier 
#model = EasyEnsembleClassifier(n_estimators=1000, random_state=1)
#model.fit(X_train_scaled, y_train)
#y_pred = model.predict(X_test_scaled)

In [44]:
model = EasyEnsembleClassifier(n_estimators=1000, random_state=1)

In [45]:
model.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=1000, random_state=1)

In [46]:
y_pred = model.predict(X_test_scaled)

In [48]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.716183869907103

In [49]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   67,    34],
       [ 3951, 13153]])

In [50]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.66      0.77      0.03      0.71      0.50       101
          1       1.00      0.77      0.66      0.87      0.71      0.52     17104

avg / total       0.99      0.77      0.66      0.86      0.71      0.52     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

1. The Easy Ensemble Classifier model has slightly higher accuracy score of 0.716.

2. The Balanced Random Forest Classifier model has better recall score of 0.83 on average, this means that the samples were classified correctly with low false negative rate.

3. The Easy Ensemble Classifier model has slightly better geometric mean of 0.71 on average.

4. The top three features that most influence the model are: total_rec_prncp, total_rec_int, last_pymnt_amnt. 
